### explore data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

xdata = pd.read_csv("./Data/YMdata.csv", header=None, usecols = range(0, 22))
ydata = pd.read_csv("./Data/YMdata.csv", header=None, usecols = range(22, 23))
xdata.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,1.22,1.42,9.6,-1.24,3.39,8.65,2.28,0.67,4.31,0.98,...,2.08,7.54,9.14,0.89,13.38,0.02,0.9,0.02,0.02,0.02
1,1.16,1.43,9.5,-2.81,6.47,7.29,4.81,1.58,4.33,1.11,...,2.16,7.82,9.07,2.17,12.73,0.05,0.8,0.05,0.05,0.05
2,1.09,1.43,9.4,-4.04,8.80,6.90,7.41,2.39,4.35,1.22,...,2.24,8.11,8.98,3.40,11.96,0.08,0.7,0.08,0.08,0.08
3,1.02,1.41,9.0,-4.44,10.21,7.28,9.81,2.74,4.22,1.26,...,2.22,8.04,9.12,4.04,11.19,0.10,0.6,0.10,0.10,0.10
4,0.94,1.39,8.6,-4.60,11.34,7.80,12.83,3.00,4.08,1.30,...,2.20,7.98,9.25,4.59,10.31,0.12,0.5,0.12,0.12,0.12


In [ ]:
# visualize the data
m = 0
for i in range(4):
    plt.figure(figsize=(14,2))
    for j in range(1, 6):
        plt.subplot(1, 5, j)
        plt.scatter(xdata[m], ydata)
        plt.xlabel(m)
        m += 1

plt.figure(figsize=(14,2))
for i in range(1, 3):
    plt.subplot(1, 5, i)
    plt.scatter(xdata[m], ydata)
    plt.xlabel(m)
    m += 1

#### columns 14 and 15 looks have no relation with YM
we will remove columns 14 and 15

In [3]:
xydata = pd.read_csv("./Data/YMdata.csv", header=None)
xydata.shape

xydata = xydata.drop(xdata.columns[[14, 15]], axis=1)
np.savetxt("./Data/selected_YMdata.csv", xydata, delimiter = ",")

### Prepare data for PyTorch

In [5]:
import torch
import torch.utils.data as td
from sklearn.model_selection import train_test_split
import pandas as pd

def loaddata(pathway, support_list, sizenum):
    df = pd.read_csv(pathway, header=None)
    x = df.iloc[:, 0:20].values
    y = df.iloc[:, 20].values

    x_support = df.iloc[support_list, 0:20].values
    y_support = df.iloc[support_list, 20].values
   
    # split data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=10)
    
    # create loader for the training data and labels
    train_x = torch.Tensor(x_train).float()
    train_x = train_x.reshape(72, 1, 20).float()
    train_y = torch.Tensor(y_train).float()

    train_data = td.TensorDataset(train_x, train_y)
    train_loader = td.DataLoader(train_data, batch_size=sizenum, shuffle=False, num_workers=1)

    # create loader for the testing data and labels
    test_x = torch.Tensor(x_test).float()
    test_x = test_x.reshape(8, 1, 20).float()
    test_y = torch.Tensor(y_test).float()

    test_data = td.TensorDataset(test_x, test_y)
    test_loader = td.DataLoader(test_data, batch_size=sizenum, shuffle=False, num_workers=1)

    # create loader for the support data and labels
    support_x = torch.Tensor(x_support).float()
    support_x = support_x.reshape(len(support_list), 1, 20).float()
    support_y = torch.Tensor(y_support).float()

    support_data = td.TensorDataset(support_x, support_y)
    support_loader = td.DataLoader(support_data, batch_size=sizenum, shuffle=False, num_workers=1)

    return train_loader, test_loader, support_loader


pathway = './Data/selected_YMdata.csv'
support_list = [0, 8, 17, 30, 49]

train_loader, test_loader, support_loader = loaddata(pathway, support_list, 8)

for data, target in train_loader:
    pass
    
print(data.shape)
print(data)



torch.Size([8, 1, 20])
tensor([[[ 1.2200e+00,  1.4200e+00,  9.6000e+00, -1.2400e+00,  3.3900e+00,
           8.6500e+00,  2.2800e+00,  6.7000e-01,  4.3100e+00,  9.8000e-01,
           2.1700e+00,  8.5100e+00,  2.0800e+00,  7.5400e+00,  1.3380e+01,
           2.0000e-02,  9.0000e-01,  2.0000e-02,  2.0000e-02,  2.0000e-02]],

        [[ 4.2000e-01,  1.0500e+00,  8.1800e+00, -3.3700e+00,  6.9900e+00,
           6.5700e+00,  3.9630e+01,  6.0000e-01,  4.2700e+00,  1.1700e+00,
           2.5600e+00,  3.3000e+00,  2.3500e+00,  8.5100e+00,  4.6100e+00,
           6.4000e-01,  8.0000e-02,  2.7000e-01,  0.0000e+00,  0.0000e+00]],

        [[ 3.3000e-01,  1.1600e+00,  8.7200e+00, -7.5000e-01,  3.3900e+00,
           1.4310e+01,  3.1130e+01,  5.9000e-01,  4.2600e+00,  9.8000e-01,
           7.4000e-01,  2.2700e+00,  2.0500e+00,  7.4500e+00,  3.6200e+00,
           9.0000e-01,  2.0000e-02,  2.0000e-02,  2.0000e-02,  2.0000e-02]],

        [[ 6.7000e-01,  1.4100e+00,  8.0400e+00, -4.0800e+00,  1.327

In [ ]:
import torch
import torch.utils.data as td
from sklearn.model_selection import train_test_split
import pandas as pd

def loaddata(pathway, support_list):
    df = pd.read_csv(pathway, header=None)
    x = df.iloc[:, 0:20].values
    y = df.iloc[:, 20].values

    x_support = df.iloc[support_list, 0:20].values
    y_support = df.iloc[support_list, 20].values
   
    # split data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=10)
    
    # create loader for the training data and labels
    train_x = torch.Tensor(x_train).float()
    train_x = train_x.reshape(72, 1, 20).float()
    train_y = torch.Tensor(y_train).float()

    train_data = td.TensorDataset(train_x, train_y)
    train_loader = td.DataLoader(train_data, batch_size=1, shuffle=False, num_workers=1)

    # create loader for the testing data and labels
    test_x = torch.Tensor(x_test).float()
    test_x = test_x.reshape(8, 1, 20).float()
    test_y = torch.Tensor(y_test).float()

    test_data = td.TensorDataset(test_x, test_y)
    test_loader = td.DataLoader(test_data, batch_size=1, shuffle=False, num_workers=1)

    # create loader for the support data and labels
    support_x = torch.Tensor(x_support).float()
    support_x = support_x.reshape(len(support_list), 1, 20).float()
    support_y = torch.Tensor(y_support).float()

    support_data = td.TensorDataset(support_x, support_y)
    support_loader = td.DataLoader(support_data, batch_size=1, shuffle=False, num_workers=1)

    return train_loader, test_loader, support_loader


pathway = './Data/selected_YMdata.csv'
support_list = [0, 8, 17, 30, 49]

train_loader, test_loader, support_loader = loaddata(pathway, support_list)

### Build training model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNNmodel(nn.Module):
    def __init__(self, inputs_num):
        super(CNNmodel, self).__init__()
        self.inputs_num = inputs_num
        self.conv0 = nn.Conv1d(2, 32, 1)
        self.conv1 = nn.Conv1d(32, 32, 1)
        self.conv2 = nn.Conv1d(32, 32, 1)
        self.conv3 = nn.Conv1d(32, 1, 1)

    def forward(self, x):
        x = F.relu(self.conv0(x))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        
        # Flatten
        x = x.view(-1, self.inputs_num)
        return x

class DNNmodel(nn.Module):
    def __init__(self, outputs_num):
        super(DNNmodel, self).__init__()
        self.outputs_num = outputs_num
        self.fc0 = nn.Linear(outputs_num, 32)
        self.fc1 = nn.Linear(32, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc0(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class FSLmodel(nn.Module):
    def __init__(self, inputs_num, x1, x2, x3, x4, x5):
        super(FSLmodel, self).__init__()
        self.x1, self.x2, self.x3, self.x4, self.x5 = x1, x2, x3, x4, x5
        self.outputs_num = inputs_num * 5
        self.Cnet = CNNmodel(inputs_num)
        self.Dnet = DNNmodel(self.outputs_num)
        
    def forward(self, x):
        
        out1 = self.Cnet(torch.cat([x, self.x1], dim=1))
        out2 = self.Cnet(torch.cat([x, self.x2], dim=1))
        out3 = self.Cnet(torch.cat([x, self.x3], dim=1))
        out4 = self.Cnet(torch.cat([x, self.x4], dim=1))
        out5 = self.Cnet(torch.cat([x, self.x5], dim=1))

        out = torch.cat([out1, out2, out3, out4, out5], dim=1)
        outputs = self.Dnet(out)
        return outputs


In [ ]:
pathway = './Data/selected_YMdata.csv'
support_list = [0, 8, 17, 30, 49]

train_loader, test_loader, support_loader = loaddata(pathway, support_list)

for datax, targetx in train_loader:
    pass
#print(datax)
#print(datax.shape)
#print(datax.shape[2])

for testx, testy in test_loader:
    pass
#print(testx)
#print(testx.shape)


sx = []
for supportx, supporty in support_loader:
    sx.append(supportx)
#print(sx[0])
#print(sx[0].shape)

inputs_num = datax.shape[2]
x1, x2, x3, x4, x5 = sx[0], sx[1], sx[2], sx[3], sx[4]
#data = torch.cat([datax, sx[0]], dim=1)
model = FSLmodel(inputs_num, x1, x2, x3, x4, x5)


In [ ]:
print(model)

### Train the model

In [ ]:
def train(model, train_loader, optimizer):
    # set model to train mode
    model.train()
    train_loss, train_error = 0, 0

    index = 0
    for data, target in train_loader:
        # feed forward
        optimizer.zero_grad()
        out = model(data)
        loss = loss_criteria(out, target)
        error = error_criteria(out, target)
        
        # backpropagate
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_error += error.item()
        index += 1

    avg_loss, avg_error = (train_loss / index), (train_error / index)
    print("Training set: Average loss: {:.6f}, error: {:.6f}".format(avg_loss, avg_error))
    return avg_loss, avg_error

def test(model, test_loader):
    # set the model to test
    model.eval()
    test_loss, error_loss = 0, 0

    with torch.no_grad():
        index = 0
        for data, target in test_loader:
            out = model(data)
            loss = loss_criteria(out, target)
            error = error_criteria(out, target)

            test_loss += loss.item()
            error_loss += error.item()
            index += 1

        avg_loss, avg_error = test_loss / index, error_loss / index
        print("Test set: Average loss: {:.6f}, error: {:.6f}".format(avg_loss, avg_error))
        return avg_loss, avg_error



#### Train the model

In [ ]:
import torch.nn as nn

# specify the loss criteria
class MyError(nn.Module):
    def __init__(self):
        super(MyError, self).__init__()

    def forward(self, pred, real):
        return torch.mean(torch.abs(torch.sub(pred, real) / real))

error_criteria = MyError()
loss_criteria = nn.MSELoss()

pathway = './Data/selected_YMdata.csv'
support_list = [0, 8, 17, 30, 49]

train_loader, test_loader, support_loader = loaddata(pathway, support_list)

sx = []
for supportx, supporty in support_loader:
    sx.append(supportx)

x1, x2, x3, x4, x5 = sx[0], sx[1], sx[2], sx[3], sx[4]
model = FSLmodel(inputs_num, x1, x2, x3, x4, x5)

# use "Adam"
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
optimizer.zero_grad()

# track meterics
epoch_nums = []
training_loss, training_error = [], []
testing_loss, testing_error = [], []

epochs = 20
for epoch in range(1, epochs+1):
    print("Epoch: {}".format(epoch))

    train_loss, train_error = train(model, train_loader, optimizer)
    test_loss, test_error = test(model, test_loader)

    epoch_nums.append(epoch)
    training_loss.append(train_loss)
    training_error.append(train_error)

    testing_loss.append(test_loss)
    testing_error.append(test_error)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(epoch_nums, training_loss)
plt.plot(epoch_nums, testing_loss)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Training', 'test'], loc='upper right')
plt.show()

#### save the trained model

In [ ]:
model_file = "./Models/pred_1VOER.pt"
torch.save(model.state_dict(), model_file)
del model
print("model is saved")

#### Use the saved model

##### Load data for prediction

In [ ]:
import torch
import torch.utils.data as td
from sklearn.model_selection import train_test_split
import pandas as pd

def loaddata(pathway1, pathway2, i, j, k):
    df1 = pd.read_csv(pathway1)
    df2 = pd.read_csv(pathway2)
    Xfeatures = ['electronegativity', 'd orbital of metal', 'group', 'radius/pm', 'first ionization energy']
    x = df1[Xfeatures].values

    label0, label1, label2 = x[i], x[j], x[k]
    label0, label1, label2 = torch.Tensor(label0).float(), torch.Tensor(label1).float(), torch.Tensor(label2).float()
    label0 = label0.reshape(1, 1, 5).float()
    label1 = label1.reshape(1, 1, 5).float()
    label2 = label2.reshape(1, 1, 5).float()
    
    predict_x = df2[Xfeatures].values
    predict_x = torch.Tensor(predict_x).float()
    #predict_x = predict_x.reshape(15, 1, 5).float()

    return predict_x, label0, label1, label2

pathway1 = "./Data/dataTrain.csv"
pathway2 = "./Data/dataTest.csv"
predict_x, label0, label1, label2 = loaddata(pathway1, pathway2, 3, 10, 4)
print(predict_x.shape)
for data in predict_x:
    pass
print(data.shape)
print(data)

for data in predict_x:
    data = data.reshape(1, 1, 5).float()
    pass
print(data.shape)
print(data)


##### predict

In [ ]:
def predict(model, predict_data, label0, label1, label2):
    # set the model to evaluation mode
    model.eval()
    predicts = []
    
    with torch.no_grad():  
        for datax in predict_data:
            datax = datax.reshape(1, 1, 5).float()
            data = torch.cat([datax, label0, label1, label2], dim=1)
            out = model(data)
            predicts.append(out.item())
        return predicts

# load data
pathway1 = "./Data/dataTrain.csv"
pathway2 = "./Data/dataTest.csv"
predictx, label0, label1, label2 = loaddata(pathway1, pathway2, 3, 4, 13)

# load model
model = CNNNet() 
model.load_state_dict(torch.load("./Models/pred_1VOER.pt"))

predicts = predict(model, predictx, label0, label1, label2)
#print(predicts)

#### save the results

In [ ]:
import numpy as np

np.savetxt("./Results/pred_1VOER.dat", np.array(predicts))